<a href="https://colab.research.google.com/github/sexymetra/Ai-project/blob/LeeSangHyuck/gru%2C%EA%B2%B0%EC%B8%A1%EC%B9%98%20%EB%B3%B4%EA%B0%84%20%EC%A0%84%EC%B2%B4%20%EB%8B%A4%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### 베이스라인은 참고용 코드 입니다. 여러분 만의 고도화된 모델을 만들어 주세요

## Import

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

#하이퍼 파라미터
CFG = {
    'EPOCHS':10,
    'LEARNING_RATE':1e-3,
    'BATCH_SIZE'
}


### 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

train_data = pd.read_csv("/content/drive/MyDrive/data/data/train_data.csv")
sample_submission = pd.read_csv("/content/drive/MyDrive/data/data/sample_submission.csv")


Mounted at /content/drive


### 데이터 검사하기
각각의 결측치들을 맞게 바꿈
filter1 전력이 0 일때는 풍속이 2.5이상일 수 없다. -> 
전력이 0이고 풍속이 2.5이상일때, 다 풍속 0 으로 바꿨음


In [ ]:
#각각의 결측치들을 맞게 바꿈
#filter1 전력이 0 일때는 풍속이 2.5이상일 수 없다. -> 전력이 0이고 풍속이 2.5이상일때, 다 풍속 0 으로 바꿨음
filter1 = (train_data['Patv']<=0) & (train_data['Wspd']>2.5)
train_data['Wspd'][filter1] = 0

#filter2 Pab1,2,3은 각각 89를 초과하면 안된다. -> Pab를 0으로 만들면 안될 것 같아서 NAN으로 만들고 있다가 보간해주기
filter2 = (train_data['Pab1']>89)|(train_data['Pab2']>89)|(train_data['Pab3']>89)
train_data['Pab1'][filter2] = np.NAN
train_data['Pab2'][filter2] = np.NAN
train_data['Pab3'][filter2] = np.NAN

#filter3 Ndir은 720을 초과하거나 -720보다 작아지면 안된다. 결측값이 별로 없으니까 720으로 나눈 나머지로 해도 될듯
filter3 = (train_data['Ndir']>720)|(train_data['Ndir']<-720)
train_data['Ndir'][filter3] = train_data['Ndir'][filter3]%720

#filter4 Wdir도 마찬가지
filter4 = (train_data['Wdir']>180)|(train_data['Wdir']<-180)
train_data['Wdir'][filter4] = train_data['Wdir'][filter4]%180


### 데이터 전처리

In [ ]:
#결측치 0으로 만들기
#train_data = train_data.fillna(0)

#3번째에 결측치를 평균으로 만들어 봄
#결측치 평균으로 만들기
train_data = train_data.interpolate(method = 'linear')
train_data = train_data.fillna(0)
#Pab1,2,3정리
train_data['Pab'] = train_data.loc[:,'Pab1':'Pab3'].median(axis=1) #Pad1,2,3의 중앙값으로 Pad설정
train_data.drop(train_data.loc[:,'Pab1':'Pab3'],axis=1,inplace = True) #Pad1,2,3 삭제
train_data = train_data.reindex(columns = ['TurbID','Day','Tmstamp','Wspd','Wdir','Etmp','Itmp','Ndir','Pab','Prtv','Patv']) #원래 순서로 행 정렬



In [ ]:
def make_train_data(data):
    train_x, train_y = [], []
    for i in tqdm(sorted(pd.unique(data["TurbID"]))):
        tmp_data = data[data["TurbID"] == i]
        for j in range(1, 201 - 6):
            
            # train data ==> 5일 단위
            # label data ==> 2일 단위
            day_list = [x for x in range(j, j+ 5)]
            label_day_list = [y for y in range(j+5, j + 7)]
            
            train_tmp = tmp_data[tmp_data["Day"].isin(day_list)]
            label_tmp = tmp_data[tmp_data["Day"].isin(label_day_list)]["Patv"]
            
            # feature 선택 및 제거
            train_tmp = train_tmp.drop(["TurbID", "Day"], axis = 1)
            
            train_x.append(np.array(train_tmp))
            train_y.append(np.array(label_tmp))
            
    return train_x, train_y

In [ ]:
# Feature 중 Tmstamp 정수로 변환
tms_list = list(pd.unique(train_data["Tmstamp"]))

train_data["Tmstamp"] = train_data["Tmstamp"].apply(lambda x : tms_list.index(x) + 1)

train_x, train_y = make_train_data(train_data)

100%|██████████| 134/134 [00:43<00:00,  3.08it/s]


In [ ]:
train_x = np.array(train_x).reshape(-1, train_x[0].shape[0], train_x[0].shape[1])
train_y = np.array(train_y)

In [ ]:
train_x.shape, train_y.shape

((25996, 720, 9), (25996, 288))

### Custom Dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        
    def __getitem__(self, index):
        if self.Y is not None:
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
        return torch.Tensor(self.X[index])
    
    def __len__(self):
        return len(self.X)

In [ ]:
train_dataset = CustomDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

### 모델 선언

In [ ]:
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.gru = nn.GRU(input_size=10, hidden_size=256, batch_first=True, bidirectional=False)
        self.classifier = nn.Sequential(
            nn.Linear(256,516),
            nn.ReLU(),
            nn.Linear(516,288),
            nn.ReLU()
        )
        
    def forward(self, x):
        hidden, _ = self.gru(x)
        output = self.classifier(hidden[:,-1,:])
        return output

### 모델 학습

In [ ]:
def train(model, optimizer, train_loader, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    metric = nn.L1Loss().to(device)#이게 L1Loss였는데 MSE로 바꿈 첫번째 버전 -> 두번째 버전
    best_mae = 9999999
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        train_mae = []
        for X, Y in tqdm(iter(train_loader)):
            X = X.to(device)
            Y = Y.to(device)
            
            optimizer.zero_grad()
            
            output = model(X)
            loss = criterion(output, Y)
            with torch.no_grad():
                mae = metric(output, Y)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
            train_mae.append(mae.item())
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Train MAE : [{np.mean(train_mae):.5f}]')
        
        if best_mae > np.mean(train_mae):
            best_mae = np.mean(train_mae)
            torch.save(model.state_dict(), './best_model.pth', _use_new_zipfile_serialization=False)
            print('Model Saved.')

In [ ]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
train(model, optimizer, train_loader, device)

  0%|          | 0/204 [00:00<?, ?it/s]


RuntimeError: ignored

### 201일, 202일 Patv 추론

In [ ]:
test_data_list = [x for x in range(196, 201)]

test_data = train_data[train_data["Day"].isin(test_data_list)]

test_data = test_data.drop(["TurbID", "Day"], axis = 1)

test_data = np.array(test_data).reshape(-1, train_x[0].shape[0], train_x[0].shape[1])

In [ ]:
test_data.shape

In [ ]:
test_dataset = CustomDataset(test_data, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def predict(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    with torch.no_grad():
        for X in tqdm(iter(test_loader)):
            X = X.to(device)
            
            pred = model(X)
            preds += pred.cpu().tolist()
    
    return np.array(preds)

### Model Load

In [ ]:
model = BaseModel()
best_checkpoint = torch.load('./best_model.pth')
model.load_state_dict(best_checkpoint)
model.eval()

In [ ]:
preds = predict(model, test_loader, device)
preds = preds.reshape(-1)

### Submit

In [ ]:
sample_submission["Patv"] = preds
sample_submission.to_csv("./submit.csv", index = False)

In [ ]:
sample_submission

In [ ]:
sample_submission['Patv'].describe()

In [ ]:
train_data['Patv'].describe()

In [ ]:
train_data['Patv'].tail(288)